## Chapter 5
# The life-cycle model and intertemporal choice

In [5]:
import numpy as np
from scipy.optimize import minimize, Bounds
from discretize_tools import *

In [6]:
class LifeCycleModel:
    '''Class of life-cycle model with 3 periods'''
    
    def __init__(self, income = 1.0,int_rate = 1.0,
                 discount_factor = 1.0, risk_aversion = 0.5,
                 n = 5, variance = 1.0, uncer_wage = False):
        self.R = int_rate
        self.β = discount_factor
        self.γ = risk_aversion
        self.egam = 1 - 1/self.γ
        self.uncer_wage = uncer_wage
        if self.uncer_wage:
            self.n = n
            self.μw = income
            self.w = np.array(log_normal_discrete(n,self.μw,variance))[0:1].reshape((n,))
            self.ω = np.array(log_normal_discrete(n,self.μw,variance))[1:].reshape((n,))
            self.a1, self.a2, self.a3 = 0, 0, np.zeros(n)
            self.c1, self.c2, self.c3 = 0, np.zeros(n), np.zeros(n)
            self.ub = np.array([self.μw] + [self.R*self.μw + self.w[i] for i in range(n)])
            self.bounds = Bounds(np.zeros(n+1)+1e-8,self.ub)
            self.guess = self.ub/2
        else:
            self.w = income
            self.ub = np.array([self.w,self.R*(self.w + 1)])
            self.bounds = Bounds(np.zeros(2),self.ub)
            self.guess = self.ub/2
            self.c1, self.c2, self.c3 = 0, 0, 0
            self.a1, self.a2, self.a3 = 0, 0, 0
    
    def __repr__(self):
        if self.uncer_wage:
            return '''Life-cycle model with uncertain wage rate:\ndisretized w: \n{} with weights\n{}\nR = {}\nβ = {}\nγ = {}'''\
               .format(self.w,self.ω,self.R,self.β,self.γ)
        else:
            return '''Life-cycle model:\nw = {}\nR = {}\nβ = {}\nγ = {}'''\
               .format(self.w,self.R,self.β,self.γ)
        
    def utility(self,x):
        if self.uncer_wage:
            self.a2 = x[0]
            self.a3 = x[1:]
            self.c1 = max(self.μw - self.a2 , 1e-10)
            self.c2 = np.maximum(self.R*self.a2+self.w-self.a3,np.full(self.n,1e-10))
            self.c3 = np.maximum(self.R*self.a3,np.full(self.n,1e-10))
            expect = .0
            for i in range(self.n):
                expect += self.ω[i]*(self.c2[i]**self.egam+self.β*self.c3[i]**self.egam)/self.egam
            return -(self.c1**self.egam/self.egam+ self.β*expect)
        else:
            self.a2 = x[0]
            self.a3 = x[1]
            self.c1 = max(self.w - self.a2, 1e-10)
            self.c2 = max(self.R*self.a2 + self.w -self.a3, 1e-10)
            self.c3 = max(self.R*self.a3, 1e-10)
            return -(self.c1**self.egam +self.β*self.c2**self.egam+self.β**2*\
                     self.c3**self.egam)/self.egam
        
    def __call__(self,i):
        if i == 1:
            return self.c1,self.a1
        elif i == 2:
            return self.c2,self.a2
        elif i == 3:
            return self.c3,self.a3
        else:
            raise VauleError('Inpyt i shoule be 1,2,or 3')
        
    def E(self,x):
        if isinstance(x,(float,int)):
            return x
        else:
            E = .0
            for i in range(len(x)):
                E += self.ω[i]*x[i]
            return E
    
    def Std(self,x):
        if isinstance(x,(float,int)):
            return 0
        else:
            std = 0.0
            for i in range(len(x)):
                std += self.ω[i]*x[i]**2
            std = (std-self.E(x)**2)**0.5
            return std
    

In [8]:
model1 = LifeCycleModel(discount_factor =.5)
minimize(model1.utility,model1.guess,bounds=model1.bounds)
print('Time  Con   Saving')
for i in range(1,4):
    print('{}     {:.2f}  {:.2f}'.format(i,model1.E(model1(i)[0]),model1.E(model1(i)[1])))

Time  Con   Saving
1     0.91  0.00
2     0.64  0.09
3     0.45  0.45


In [4]:
model2 = LifeCycleModel(uncer_wage = True)
minimize(model2.utility,model2.guess,bounds=model2.bounds)
print('Time  Con   Saving')
for i in range(1,4):
    print('{}     {:.2f}  {:.2f} (Mean)'.format(i,model2.E(model2(i)[0]),model2.E(model2(i)[1])))
    print('      {:.2f}  {:.2f} (Std)'.format(model2.Std(model2(i)[0]),model2.Std(model2(i)[1])))

Time  Con   Saving
1     0.53  0.00 (Mean)
      0.00  0.00 (Std)
2     0.74  0.47 (Mean)
      0.50  0.00 (Std)
3     0.74  0.74 (Mean)
      0.50  0.50 (Std)
